# Using DataYoink's Pretrained Mask

This demo is inteded for users who have images of battery discharge curves from which they want to extract points. This can be accomplished easily by following the steps below in a jupyter notebook.

## Step 1: Import all requred packages

The import statements below may need to be modified based on where the requred .py files appear in your directory

In [3]:
from pyfiles_for_demos.coordconverter import datayoink_to_excel, get_axis_info

## Step 2:  Load and resize Image

## Step 3: Provide information about the image

## Step 4: Run detectron2 using pretrained model

## Step 5. Use DataYoink to save coordinates to excel